# FullControl design template

*<<< check out demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*
  
run all cells in this notebook, or press shift+enter to run each cell sequentially 

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*

In [1]:
import fullcontrol as fc
import math

In [2]:
def circle_infill(diameter:float,start_point:fc.Point,line_space:float,direction:float):
    '''
    Generates a square wave with the number of periods and amplitude bound by the diameter of a circle, 
    with space between periods defined by line_space, staring at XY point (start_point) 
    along a line with the polar angle given by direction. Returns a list of points

    Parameters:
    - diameter (float): Diameter of the circle
    - start_point (Point): The starting point of the square wave.       
    - line_space (float): The distance between each line of the square wave.
    - direction (float): The angle in polar cooridinates of the vector the square wave moves.    
    
    Returns:
    - list: A list of Points representing the square wave.  
    '''
    import math
    import numpy as np
    radius_c=diameter/2
    steps=[]
    flag=True
    steps.append(start_point)
    steps.append(fc.polar_to_point(centre=steps[-1],radius=line_space,angle=direction))
    for space in np.arange(line_space,diameter,line_space):
        if flag:
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction+math.radians(90)))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=line_space,angle=direction))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction-math.radians(90)))
        else:
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction-math.radians(90)))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=line_space,angle=direction))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction+math.radians(90)))            
        flag=not flag       
    return steps

In [28]:
def circle_infill_arc(diameter:float,start_point:fc.Point,centre_point:fc.Point,line_space:float,direction:float):
    import math
    import numpy as np
    radius_c=diameter/2    
    steps=[]
    flag=1
    start_polar=fc.point_to_polar(start_point,centre_point)
    for space in np.arange(0,diameter,line_space):
                start_angle=math.acos((radius_c-space)/radius_c)+start_polar.angle
                end_angle=math.acos((radius_c-(space+line_space))/radius_c) +start_polar.angle                       
                steps.extend(fc.arcXY(centre=centre_point, radius=radius_c,start_angle=(start_angle*flag)+direction,arc_angle=(end_angle-start_angle)*flag,segments=32))
                #steps.append(fc.polar_to_point(centre=steps[-1], radius=(math.sqrt((radius_c**2)-(radius_c-space)**2)),angle=start_polar.angle-(math.radians(90)*flag)))           
                flag = -flag       
    return steps

In [4]:
# printer/gcode parameters

design_name = 'cylinder'
nozzle_temp = 0
bed_temp = 0
print_speed = 300
fan_percent = 0
printer_name='stoneflower3d' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [44]:
# design parameters

EW = 0.5 # extrusion width
EH = 0.25 # extrusion height (and layer height)
initial_z = EH*1 # initial nozzle position can be set to a multiple of the extrusion height to get a bit of 'squish' for good bed adhesion
height=1 #Height of cylinder in mm
layers = int(height//EH)
dia=20 #diameter of the cylinder in mm
radius = dia/2


In [45]:
# generate the design (make sure you've run the above cells before running this cell)
initial_xyz=fc.Point(x=200, y=200, z=initial_z)
steps = []
#Generate the layer patterns such that no travel is require between the layers
for layer in range(layers):
        centre_point = fc.move(initial_xyz,fc.Vector(x=radius,y=radius,z=EH*layer))      
        
        layer1=[]
        first_point=fc.move(initial_xyz,fc.Vector(y=radius,z=EH*layer))
        start_angle =fc.point_to_polar(first_point,centre_point)
        circle_start=start_angle.angle+math.pi
        segments = 32
        clockwise = False
        line_spacing = 1 
        circle_steps = fc.circleXY(centre_point, radius, circle_start, segments, clockwise)#calculate perimeter points
        circle_steps.extend(fc.arcXY(centre_point,radius-EW,circle_start,0.5*math.pi,segments))#calculate inner arc to move nozzle inposition for next layer        
        first_point=fc.move(initial_xyz,fc.Vector(y=radius,z=EH*layer))               
        infill_steps = circle_infill_arc(dia-4*EW,first_point,centre_point,line_spacing,0)#Calculate infill points              
        layer1.extend(infill_steps)#Draw the infill first
        layer1.extend(circle_steps)#Draw perimeter and arc       
           
    
        layer2=[] # same as previous layer but offset by 90 degrees about the center       
        first_point=fc.move(initial_xyz,fc.Vector(x=dia,y=radius,z=EH*layer))
        start_angle =fc.point_to_polar(first_point,centre_point)
        print(start_angle)
        circle_start=start_angle.angle-math.pi/2
        segments = 32
        clockwise = False
        line_spacing = 1         
        circle_steps = fc.circleXY(centre_point, radius, circle_start, segments, clockwise)#calculate perimeter points
        circle_steps.extend(fc.arcXY(centre_point,radius-EW,circle_start,-0.5*math.pi,segments))#calculate inner arc to move nozzle inposition for next layer        
        infill_steps = circle_infill_arc(dia-4*EW,first_point,centre_point,line_spacing,math.pi/2)       
        layer2.extend(infill_steps)
        layer2.extend(circle_steps)
       
        
        layertypes=[layer1,layer2]
        steps.extend(layertypes[layer%len(layertypes)])#Alternate drawing between layer1 and layer2 



radius=10.0 angle=0.0
radius=10.0 angle=0.0
radius=10.0 angle=0.0
radius=10.0 angle=0.0


In [46]:
# preview the design

fc.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.7))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
fc.transform(steps, 'plot', fc.PlotControls(style='tube', zoom=0.7, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.5, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


    - use fc.transform(..., controls=fc.PlotControls(style='tube') to disable this message or style='line' for a simpler line preview



In [ ]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,
    save_as=design_name,
    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109. 
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol